In [1]:
import sys
import os
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(parent_dir)

from sklearn import preprocessing
import numpy as np
import pickle as pkl
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow import keras
from utils.dataset_utils import batch_generator 
from DANN_trainer import DANN
from utils.LoadData import load_CW_Source,load_CW_Target_validation
from utils.DANN_config import config



d:\VirtualEnvironment\Anaconda3\envs\TF-krease\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
d:\VirtualEnvironment\Anaconda3\envs\TF-krease\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
d:\VirtualEnvironment\Anaconda3\envs\TF-krease\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
d:\VirtualEnvironment\Anaconda3\envs\TF-krease\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
d:\VirtualEnvironment\Anaconda3\envs\TF-krease\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
gpus = tf.config.experimental.list_physical_devices(device_type='GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [3]:
def run_main():
    """Main function for side-channel analysis using domain adaptation"""
    # Initialize power trace parameters
    power_input_shape = (700, 1)  # Original trace length and channels
    trace_length = 700           # Renamed from image_size
    init_learning_rate = 5e-3
    momentum_rate = 0.9
    batch_size = 2000
    epochs = 60
    pre_trained_path = None
    output_path = './Model/'
    profiling_Data_path='./Dataset/AES_OriginalMDS/'
    Target_Data_path='./Dataset/AES_INVMDS/'
 
    # Configure paths for SCA results
    checkpoints_dir = os.path.abspath(output_path)
        
     # Create configuration object
    sca_config=config(
            pre_trained_path=pre_trained_path,
            checkpoints_dir=checkpoints_dir,
            power_input_shape=power_input_shape,
            trace_length=trace_length,
            init_learning_rate=init_learning_rate,
            momentum_rate=momentum_rate,
            batch_size=batch_size,
            epochs=epochs
    )

    # Load source device power traces
    src_traces_train, src_labels_train,src_traces_val, src_labels_val,_,_ = load_CW_Source(
            in_file=profiling_Data_path,
            sec=18000,  # Fixed security parameter from original implementation
    )
    src_labels_train=src_labels_train[:,0]
    src_labels_val= src_labels_val[:,0]
        
    # Convert labels for SCA classification (256 classes for byte values)
    src_labels_train = to_categorical(src_labels_train).astype(np.float32)
    src_labels_val = to_categorical(src_labels_val).astype(np.float32)

    # Preprocess power traces
    scaler = preprocessing.StandardScaler()
    src_traces_train = scaler.fit_transform(src_traces_train)
    src_traces_val = scaler.transform(src_traces_val)

    # Load target device traces
    # Note: Target domain labels are only used for validation accuracy, not for gradient updates
    tgt_traces_train, tgt_labels_train, tgt_traces_val, tgt_labels_val, _,_ = load_CW_Target_validation(
            in_file=Target_Data_path,
            sec=18000,  # Fixed security parameter from original implementation
    )
    tgt_labels_train=tgt_labels_train[:,0]
    tgt_labels_val=tgt_labels_val[:,0]

    # Process target device data
    tgt_labels_train = to_categorical(tgt_labels_train).astype(np.float32)
    tgt_labels_val = to_categorical(tgt_labels_val).astype(np.float32)
    tgt_traces_train = scaler.transform(tgt_traces_train)
    tgt_traces_val = scaler.transform(tgt_traces_val)

    # Reshape traces for CNN input
    src_traces_train = src_traces_train.reshape((-1, power_input_shape[0], 1))
    tgt_traces_train = tgt_traces_train.reshape((-1, power_input_shape[0], 1))
    tgt_traces_val = tgt_traces_val.reshape((-1, power_input_shape[0], 1))

    # Create trace data generators
    train_source_gen = batch_generator(
        [src_traces_train, src_labels_train], 
        batch_size=batch_size//2
    )
    train_target_gen = batch_generator(
        [tgt_traces_train, tgt_labels_train],
        batch_size=batch_size//2,
        shuffle=False
    )
    val_target_gen = batch_generator(
        [tgt_traces_val, tgt_labels_val],
        batch_size=batch_size
    )

    # Calculate training iterations
    train_iters = max(
        len(src_traces_train) // (batch_size//2),
        len(tgt_traces_train) // (batch_size//2)
    )
    val_iters = len(tgt_traces_val) // batch_size

    # Initialize fresh model session
    tf.keras.backend.clear_session()

    # Initialize and train SCA model
    sca_model = DANN(sca_config)
    sca_model.train(train_source_gen,train_target_gen,val_target_gen,train_iters,val_iters)

In [4]:
if __name__ == '__main__':
    run_main()


----------- start to train -----------

Epoch 1/50

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

19/19 [==============================] - 3s 151ms/step - loss: 5.1961 - source_train_loss: 5.0447 - domain_train_loss: 0.1514 - source_train_acc: 0.0541 - domain_train_acc: 0.9712 - target_loss: 14.5410 - target_acc: 0.0033
Epoch001-train_loss-5.196-val_loss-14.541-train_image_cls_acc-0.054-val_image_cls_acc-0.003
Epoch 2/50
19/19 [==============================] - 3s 152ms/step - loss: 2.5270 - source_train_loss: 2.4099 - domain_train_loss: 0.1171 - s